In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [7]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels)) # real loss
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    ###################################################### ~Q(s,~a)= r # min
    ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # minimize Q
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # maximize Q
    eloss += tf.reduce_mean(targetQs) # minimize Q
    aloss2 += -tf.reduce_mean(targetQs) # maxizmie Q
    eloss += tf.reduce_mean(eQs_logits) # minimize Q
    aloss2 += -tf.reduce_mean(eQs_logits) # maxizmie Q
    return actions_logits, aloss, eloss, aloss2

In [8]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [10]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [11]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [12]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [13]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [14]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:19.0000 R:19.0000 rate:0.0380 aloss:0.6972 eloss:5.6391 aloss2:1.2661 exploreP:0.9981
Episode:1 meanR:19.5000 R:20.0000 rate:0.0400 aloss:0.6947 eloss:5.6019 aloss2:1.2496 exploreP:0.9961
Episode:2 meanR:18.3333 R:16.0000 rate:0.0320 aloss:0.6970 eloss:5.5963 aloss2:1.2870 exploreP:0.9946
Episode:3 meanR:18.7500 R:20.0000 rate:0.0400 aloss:0.6959 eloss:5.5603 aloss2:1.3102 exploreP:0.9926
Episode:4 meanR:20.6000 R:28.0000 rate:0.0560 aloss:0.6942 eloss:5.5766 aloss2:1.2999 exploreP:0.9899
Episode:5 meanR:20.5000 R:20.0000 rate:0.0400 aloss:0.6947 eloss:5.5906 aloss2:1.3086 exploreP:0.9879
Episode:6 meanR:20.2857 R:19.0000 rate:0.0380 aloss:0.6945 eloss:5.4196 aloss2:1.4772 exploreP:0.9860
Episode:7 meanR:19.5000 R:14.0000 rate:0.0280 aloss:0.6925 eloss:5.5164 aloss2:1.3371 exploreP:0.9847
Episode:8 meanR:20.4444 R:28.0000 rate:0.0560 aloss:0.6922 eloss:5.4242 aloss2:1.4505 exploreP:0.9820
Episode:9 meanR:25.9000 R:75.0000 rate:0.1500 aloss:0.6929 eloss:5.5060 aloss2:1.3

Episode:80 meanR:26.7901 R:45.0000 rate:0.0900 aloss:0.7171 eloss:4.9157 aloss2:2.7768 exploreP:0.8069
Episode:81 meanR:26.6463 R:15.0000 rate:0.0300 aloss:0.6915 eloss:4.8840 aloss2:2.7902 exploreP:0.8057
Episode:82 meanR:26.6024 R:23.0000 rate:0.0460 aloss:0.7274 eloss:4.9255 aloss2:2.8988 exploreP:0.8039
Episode:83 meanR:26.5476 R:22.0000 rate:0.0440 aloss:0.7078 eloss:4.8924 aloss2:2.8582 exploreP:0.8021
Episode:84 meanR:26.7647 R:45.0000 rate:0.0900 aloss:0.6985 eloss:4.8976 aloss2:2.8696 exploreP:0.7986
Episode:85 meanR:26.6628 R:18.0000 rate:0.0360 aloss:0.7121 eloss:4.9210 aloss2:2.7907 exploreP:0.7971
Episode:86 meanR:26.5057 R:13.0000 rate:0.0260 aloss:0.8002 eloss:4.9286 aloss2:2.9602 exploreP:0.7961
Episode:87 meanR:26.3636 R:14.0000 rate:0.0280 aloss:0.7173 eloss:4.8989 aloss2:2.8573 exploreP:0.7950
Episode:88 meanR:26.2135 R:13.0000 rate:0.0260 aloss:0.7024 eloss:4.8914 aloss2:2.9187 exploreP:0.7940
Episode:89 meanR:26.9889 R:96.0000 rate:0.1920 aloss:0.7221 eloss:4.9088 

Episode:159 meanR:39.3500 R:52.0000 rate:0.1040 aloss:0.7876 eloss:4.7741 aloss2:3.6131 exploreP:0.5851
Episode:160 meanR:39.4000 R:51.0000 rate:0.1020 aloss:0.7466 eloss:4.7722 aloss2:3.6058 exploreP:0.5822
Episode:161 meanR:39.4100 R:17.0000 rate:0.0340 aloss:0.7569 eloss:4.7656 aloss2:3.6771 exploreP:0.5812
Episode:162 meanR:39.0700 R:33.0000 rate:0.0660 aloss:0.7506 eloss:4.7742 aloss2:3.6187 exploreP:0.5793
Episode:163 meanR:39.0900 R:49.0000 rate:0.0980 aloss:0.7465 eloss:4.7771 aloss2:3.5996 exploreP:0.5765
Episode:164 meanR:39.2500 R:60.0000 rate:0.1200 aloss:0.7925 eloss:4.7758 aloss2:3.6026 exploreP:0.5731
Episode:165 meanR:39.5600 R:45.0000 rate:0.0900 aloss:0.7609 eloss:4.7660 aloss2:3.5916 exploreP:0.5706
Episode:166 meanR:39.1500 R:13.0000 rate:0.0260 aloss:0.6870 eloss:4.7570 aloss2:3.7266 exploreP:0.5699
Episode:167 meanR:39.5800 R:58.0000 rate:0.1160 aloss:0.7500 eloss:4.7604 aloss2:3.6726 exploreP:0.5666
Episode:168 meanR:39.5300 R:35.0000 rate:0.0700 aloss:0.7510 elo

Episode:238 meanR:46.6500 R:26.0000 rate:0.0520 aloss:0.6964 eloss:4.7687 aloss2:3.6691 exploreP:0.4062
Episode:239 meanR:46.4900 R:26.0000 rate:0.0520 aloss:0.7060 eloss:4.7671 aloss2:3.6419 exploreP:0.4052
Episode:240 meanR:46.4300 R:23.0000 rate:0.0460 aloss:0.7106 eloss:4.7846 aloss2:3.6417 exploreP:0.4043
Episode:241 meanR:46.1800 R:11.0000 rate:0.0220 aloss:0.7207 eloss:4.7618 aloss2:3.5888 exploreP:0.4039
Episode:242 meanR:45.9500 R:37.0000 rate:0.0740 aloss:0.7001 eloss:4.7948 aloss2:3.6052 exploreP:0.4024
Episode:243 meanR:45.3400 R:28.0000 rate:0.0560 aloss:0.7246 eloss:4.7699 aloss2:3.5359 exploreP:0.4013
Episode:244 meanR:45.0800 R:27.0000 rate:0.0540 aloss:0.6930 eloss:4.7640 aloss2:3.5571 exploreP:0.4003
Episode:245 meanR:44.8900 R:17.0000 rate:0.0340 aloss:0.7180 eloss:4.7630 aloss2:3.5607 exploreP:0.3996
Episode:246 meanR:44.9000 R:38.0000 rate:0.0760 aloss:0.6917 eloss:4.7620 aloss2:3.5757 exploreP:0.3981
Episode:247 meanR:44.8600 R:44.0000 rate:0.0880 aloss:0.6909 elo

Episode:317 meanR:22.2100 R:10.0000 rate:0.0200 aloss:0.7539 eloss:4.7976 aloss2:3.6431 exploreP:0.3553
Episode:318 meanR:21.6600 R:10.0000 rate:0.0200 aloss:0.7816 eloss:4.8043 aloss2:3.5945 exploreP:0.3550
Episode:319 meanR:21.1800 R:11.0000 rate:0.0220 aloss:0.7422 eloss:4.8594 aloss2:3.6879 exploreP:0.3546
Episode:320 meanR:20.6600 R:11.0000 rate:0.0220 aloss:0.7360 eloss:4.7825 aloss2:3.6081 exploreP:0.3542
Episode:321 meanR:20.4600 R:20.0000 rate:0.0400 aloss:0.7419 eloss:4.7758 aloss2:3.6402 exploreP:0.3535
Episode:322 meanR:20.0700 R:12.0000 rate:0.0240 aloss:0.7488 eloss:4.7786 aloss2:3.6391 exploreP:0.3531
Episode:323 meanR:19.4900 R:8.0000 rate:0.0160 aloss:0.7478 eloss:4.7719 aloss2:3.6837 exploreP:0.3529
Episode:324 meanR:18.9000 R:18.0000 rate:0.0360 aloss:0.7296 eloss:4.7777 aloss2:3.6390 exploreP:0.3522
Episode:325 meanR:18.2400 R:10.0000 rate:0.0200 aloss:0.7321 eloss:4.7817 aloss2:3.6264 exploreP:0.3519
Episode:326 meanR:17.9400 R:12.0000 rate:0.0240 aloss:0.7383 elos

...

Episode:871 meanR:35.7200 R:10.0000 rate:0.0200 aloss:0.6647 eloss:4.6672 aloss2:4.0163 exploreP:0.1017
Episode:872 meanR:35.4000 R:12.0000 rate:0.0240 aloss:0.7175 eloss:4.6818 aloss2:4.0412 exploreP:0.1016
Episode:873 meanR:35.0600 R:10.0000 rate:0.0200 aloss:0.7215 eloss:4.7118 aloss2:4.0799 exploreP:0.1015
Episode:874 meanR:34.3000 R:37.0000 rate:0.0740 aloss:0.7245 eloss:4.6522 aloss2:4.1126 exploreP:0.1012
Episode:875 meanR:32.3900 R:9.0000 rate:0.0180 aloss:0.7262 eloss:4.6369 aloss2:4.1146 exploreP:0.1011
Episode:876 meanR:31.6700 R:11.0000 rate:0.0220 aloss:0.7301 eloss:4.6324 aloss2:4.0987 exploreP:0.1010
Episode:877 meanR:31.0800 R:11.0000 rate:0.0220 aloss:0.7094 eloss:4.6406 aloss2:4.1238 exploreP:0.1009
Episode:878 meanR:30.5100 R:10.0000 rate:0.0200 aloss:0.7314 eloss:4.6158 aloss2:4.1425 exploreP:0.1008
Episode:879 meanR:29.9000 R:11.0000 rate:0.0220 aloss:0.7449 eloss:4.5563 aloss2:4.2421 exploreP:0.1007
Episode:880 meanR:29.3600 R:12.0000 rate:0.0240 aloss:0.7099 elos

Episode:951 meanR:15.6000 R:8.0000 rate:0.0160 aloss:0.7961 eloss:4.9551 aloss2:3.7166 exploreP:0.0909
Episode:952 meanR:15.5500 R:8.0000 rate:0.0160 aloss:0.7822 eloss:4.8704 aloss2:3.8307 exploreP:0.0909
Episode:953 meanR:15.5600 R:10.0000 rate:0.0200 aloss:0.8229 eloss:4.9665 aloss2:3.7063 exploreP:0.0908
Episode:954 meanR:15.1300 R:9.0000 rate:0.0180 aloss:0.8175 eloss:4.9116 aloss2:3.7178 exploreP:0.0907
Episode:955 meanR:15.1100 R:9.0000 rate:0.0180 aloss:0.8147 eloss:4.9010 aloss2:3.7270 exploreP:0.0906
Episode:956 meanR:15.1000 R:9.0000 rate:0.0180 aloss:0.7693 eloss:4.9377 aloss2:3.7539 exploreP:0.0906
Episode:957 meanR:15.1100 R:12.0000 rate:0.0240 aloss:0.7543 eloss:4.9213 aloss2:3.7948 exploreP:0.0905
Episode:958 meanR:15.1100 R:11.0000 rate:0.0220 aloss:0.8119 eloss:4.9748 aloss2:3.6904 exploreP:0.0904
Episode:959 meanR:15.1100 R:9.0000 rate:0.0180 aloss:0.7754 eloss:4.8815 aloss2:3.8126 exploreP:0.0903
Episode:960 meanR:15.1100 R:8.0000 rate:0.0160 aloss:0.7658 eloss:4.91

Episode:1030 meanR:13.5600 R:10.0000 rate:0.0200 aloss:0.7438 eloss:4.8510 aloss2:3.4867 exploreP:0.0825
Episode:1031 meanR:13.5700 R:27.0000 rate:0.0540 aloss:0.7315 eloss:4.8819 aloss2:3.4455 exploreP:0.0823
Episode:1032 meanR:13.5200 R:9.0000 rate:0.0180 aloss:0.7236 eloss:4.8638 aloss2:3.4053 exploreP:0.0822
Episode:1033 meanR:13.5700 R:18.0000 rate:0.0360 aloss:0.7432 eloss:4.8191 aloss2:3.5441 exploreP:0.0821
Episode:1034 meanR:13.6800 R:22.0000 rate:0.0440 aloss:0.7146 eloss:4.8288 aloss2:3.5485 exploreP:0.0819
Episode:1035 meanR:13.6700 R:10.0000 rate:0.0200 aloss:0.7138 eloss:4.8356 aloss2:3.5626 exploreP:0.0818
Episode:1036 meanR:13.7800 R:36.0000 rate:0.0720 aloss:0.7181 eloss:4.8384 aloss2:3.4848 exploreP:0.0816
Episode:1037 meanR:14.2100 R:51.0000 rate:0.1020 aloss:0.7051 eloss:4.8365 aloss2:3.5050 exploreP:0.0812
Episode:1038 meanR:14.5300 R:41.0000 rate:0.0820 aloss:0.7097 eloss:4.8323 aloss2:3.4943 exploreP:0.0809
Episode:1039 meanR:14.7700 R:34.0000 rate:0.0680 aloss:0

Episode:1109 meanR:28.7000 R:11.0000 rate:0.0220 aloss:0.7372 eloss:4.7947 aloss2:3.6570 exploreP:0.0660
Episode:1110 meanR:29.2300 R:66.0000 rate:0.1320 aloss:0.7360 eloss:4.8217 aloss2:3.5965 exploreP:0.0656
Episode:1111 meanR:29.2600 R:12.0000 rate:0.0240 aloss:0.6987 eloss:4.8062 aloss2:3.6102 exploreP:0.0655
Episode:1112 meanR:29.7000 R:65.0000 rate:0.1300 aloss:0.7065 eloss:4.8194 aloss2:3.6008 exploreP:0.0652
Episode:1113 meanR:30.0600 R:45.0000 rate:0.0900 aloss:0.7224 eloss:4.8215 aloss2:3.5697 exploreP:0.0649
Episode:1114 meanR:30.1000 R:13.0000 rate:0.0260 aloss:0.6697 eloss:4.7880 aloss2:3.6100 exploreP:0.0648
Episode:1115 meanR:30.5000 R:50.0000 rate:0.1000 aloss:0.7138 eloss:4.8206 aloss2:3.5395 exploreP:0.0646
Episode:1116 meanR:31.0000 R:60.0000 rate:0.1200 aloss:0.7155 eloss:4.8199 aloss2:3.5499 exploreP:0.0642
Episode:1117 meanR:32.1900 R:129.0000 rate:0.2580 aloss:0.7063 eloss:4.8172 aloss2:3.5443 exploreP:0.0635
Episode:1118 meanR:32.8600 R:77.0000 rate:0.1540 aloss

Episode:1187 meanR:104.9400 R:368.0000 rate:0.7360 aloss:0.7277 eloss:4.7730 aloss2:3.5546 exploreP:0.0315
Episode:1188 meanR:104.9600 R:102.0000 rate:0.2040 aloss:0.7065 eloss:4.7837 aloss2:3.4986 exploreP:0.0313
Episode:1189 meanR:104.0900 R:79.0000 rate:0.1580 aloss:0.7317 eloss:4.7773 aloss2:3.4757 exploreP:0.0312
Episode:1190 meanR:103.8000 R:56.0000 rate:0.1120 aloss:0.7031 eloss:4.7729 aloss2:3.4875 exploreP:0.0310
Episode:1191 meanR:104.1500 R:121.0000 rate:0.2420 aloss:0.7097 eloss:4.7744 aloss2:3.4713 exploreP:0.0308
Episode:1192 meanR:104.1700 R:58.0000 rate:0.1160 aloss:0.7450 eloss:4.7681 aloss2:3.4680 exploreP:0.0307
Episode:1193 meanR:104.3500 R:77.0000 rate:0.1540 aloss:0.6716 eloss:4.7543 aloss2:3.4822 exploreP:0.0305
Episode:1194 meanR:104.8800 R:108.0000 rate:0.2160 aloss:0.6842 eloss:4.7623 aloss2:3.4640 exploreP:0.0303
Episode:1195 meanR:105.8500 R:171.0000 rate:0.3420 aloss:0.6689 eloss:4.7598 aloss2:3.4597 exploreP:0.0299
Episode:1196 meanR:107.2500 R:152.0000 ra

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
